<a href="https://colab.research.google.com/github/AUT-Student/NLP-HW2/blob/main/NLP_HW2_Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [7]:
import pandas as pd
import numpy as np

# Dataset

In [2]:
!gdown 1-86CqCHek-U1iH5nW30RfnFU0PYmdhKB

Downloading...
From: https://drive.google.com/uc?id=1-86CqCHek-U1iH5nW30RfnFU0PYmdhKB
To: /content/train.csv
100% 20.1M/20.1M [00:00<00:00, 224MB/s]


In [4]:
!gdown 1YzRlYyye_KoEw7_q9NARiCwl3Cn-EH3J

Downloading...
From: https://drive.google.com/uc?id=1YzRlYyye_KoEw7_q9NARiCwl3Cn-EH3J
To: /content/test.csv
100% 2.03M/2.03M [00:00<00:00, 148MB/s]


In [10]:
train_data = pd.read_csv("/content/train.csv")
test_data = pd.read_csv("/content/test.csv")